# 웹 게시글 알람서비스 만들기 (Python)


특정 웹사이트에서 내가 설정해둔 키워드로 글이 등록되었을 때

메시지를 전송해주는 파이썬 프로그램을 만들어 보겠습니다.

특가 정보, 관심기업 정보, 관심분야 정보 등에 활용해봅시다.

---

### 01. 라이브러리 설치

In [ ]:
# !pip install selenium

In [ ]:
# !pip install webdriver-manager

In [ ]:
# !pip install python-telegram-bot

- selenium : 웹 제어 라이브러리. 크롤링에 주로 사용됩니다.
- webdriver-manager : 크롬 드라이버 사용을 위해 설치합니다.
- python-telegram-bot : 텔레그램을 사용한 메시지 전송이 가능해집니다.

---

### 2. 특정 사이트 게시판 가져오기

In [17]:
from selenium import webdriver

In [18]:
from selenium.webdriver.common.by import By

In [19]:
# selenium 버전 확인
import selenium
selenium.__version__

'4.5.0'

- 최근 selenium 4.3 버전부터 element 가져오는 방법이 변경되었습니다.
- find_by_css_selector -> find_element(By.CSS_SELECTOR)

In [20]:
from webdriver_manager.chrome import ChromeDriverManager

In [22]:
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get(url="https://ikaos.org/kaos/apply/list.php")

driver.implicitly_wait(time_to_wait=10)

C:\Users\21csc\AppData\Local\Temp\ipykernel_15260\2072065792.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


카오스 강연 게시판을 가져왔습니다.
#container > div > div.interview-list > div.book-list.type2 > div.post-list > div.book-list.type2 > ul > li > a > div.book-info > div:nth-child(2) > strong
#container > div > div.interview-list > div.book-list.type2 > div.post-list > div.book-list.type2 > ul > li:nth-child(2) > a > div.book-info > div:nth-child(2) > strong

---

### 3. 게시글 가져오기

driver.find_elements(By.CSS_SELECTOR, "") 사용법
- 웹의 가져올 부분 > 우클릭 > 검사 > 해당 태그 우클릭 > Copy > Copy Selector
- (CSS_SELECTOR 이외의 기능을 사용할 수도 있습니다. 다양한 옵션을 Copy 해보면 좋을 것 같습니다.)
- 각각의 제목을 Copy Selector 후 값을 확인하여 규칙성을 발견합니다.
- 각 제목의 순서를 결정하는 부분을 제거하여, 모두 가져올 수 있도록 합니다.

In [23]:
titles = driver.find_elements(By.CSS_SELECTOR, "#container > div > div.interview-list > div.book-list.type2 > div.post-list > div.book-list.type2 > ul > li > a > div.book-info > div:nth-child(2) > strong")

In [24]:
urls = driver.find_elements(By.CSS_SELECTOR, "#container > div > div.interview-list > div.book-list.type2 > div.post-list > div.book-list.type2 > ul > li > a")

In [25]:
for i in range(len(titles)):
    print(titles[i].text)
    print(urls[i].get_attribute("href"))

2022 노벨상 해설 강연
https://ikaos.org/kaos/apply/view.php?kc_idx=147
[무료상영] 트리오브라이프
https://ikaos.org/kaos/apply/view.php?kc_idx=146
진화
https://ikaos.org/kaos/apply/view.php?kc_idx=145
카오스+어스 ㅣ 페스티벌 어스 프리퀄 행사
https://ikaos.org/kaos/apply/view.php?kc_idx=144
[식물행성] 10강(5/18) 출석체크
https://ikaos.org/kaos/apply/view.php?kc_idx=142
[식물행성] 9강(5/11) 출석체크
https://ikaos.org/kaos/apply/view.php?kc_idx=141
생명행성 Life Planet
https://ikaos.org/kaos/apply/view.php?kc_idx=140


---

### 4. Telegram api로 메시지 전송

여러가지 메시지 전송방법이 있지만 (카카오 API 활용 등)

가장 간단하게 테스트 할 수 있는 텔레그램을 사용했습니다.



<img src="https://ifh.cc/g/6M8R0g.png" width="400" align="left"/>

[Telegram api 토큰, id 얻는 방법]
- 텔레그램 다운로드 및 로그인 > 사용자 검색 > "BotFather" 검색 > 채팅 시작
- "/start" 입력 > "/newbot" 클릭
- Bot id 입력하기 (Bot id는 "_bot" 으로 끝나야 한다.)
- 사용 가능하면, username 입력하기 (마찬가지로 "_bot" 으로 끝나야 한다.)
- User token이 발급된다.

In [26]:
import telegram

In [27]:
# 텔레그램 id 확인을 위한 코드입니다.

token = '5459255337:AAGwy804XtrZySObftsXT_pBsohT4Ik5-64'
bot = telegram.Bot(token=token)
updates = bot.getUpdates()
for u in updates:
    print(u.message)

In [28]:
message_1st = titles[0].text
print(message_1st)

2022 노벨상 해설 강연


In [29]:
message = ""
for i in range(len(titles)):
    if "강연" in titles[i].text:
        message = titles[i].text
        print(message)
        
        token = "5459255337:AAGwy804XtrZySObftsXT_pBsohT4Ik5-64"
        id = "5332455701"
        bot = telegram.Bot(token)
        bot.sendMessage(chat_id=id, text=message)

2022 노벨상 해설 강연


"강연" 키워드가 포함된 타이틀을 가져와 메시지 전송을 합니다.

---

### 5. 일정 주기로 확인 후 메시지를 전송하기
- .py 파일을 생성하여 아래 코드를 작성 후 실행합니다.
- time.sleep 입력값에 따라 일정한 주기로 코드가 실행됩니다.

In [19]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
import telegram
import time

driver = webdriver.Chrome(ChromeDriverManager().install())

send_message_list = []

while True:
    try:
        driver.get(url="https://ikaos.org/kaos/apply/list.php")
        
        titles = driver.find_elements(By.CSS_SELECTOR, "#container > div > div.interview-list > div.book-list.type2 > div.post-list > div.book-list.type2 > ul > li > a > div.book-info > div:nth-child(2) > strong")
        urls = driver.find_elements(By.CSS_SELECTOR, "#container > div > div.interview-list > div.book-list.type2 > div.post-list > div.book-list.type2 > ul > li > a")
        
        message = ""
        
        for i in range(len(titles)):
            if "강연" in titles[i].text:
                message = titles[i].text + "\n" + urls[i].get_attribute("href")
                
                if message not in send_message_list:
                    send_message_list.append(message)
                    
                    token = "5459255337:AAGwy804XtrZySObftsXT_pBsohT4Ik5-64"
                    id = "5332455701"
                    bot = telegram.Bot(token)
                    bot.sendMessage(chat_id=id, text=message)
                    
        time.sleep(60.0)
        
    except KeyboardInterrupt:
        break

C:\Users\21csc\AppData\Local\Temp\ipykernel_17020\3654725657.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


다른 방법 시도하기
- 이마트와 홈플러스의 빙그레바나나맛우유 가격을 비교하여 알람 서비스를 만들어보자.
- 아이폰14 pro 가격이 100만원 이하가 되었을 때 알람을 받아보자.

---

### 6. 이미지를 가져와 텍스트 추출하기

- 텍스트 추출 라이브러리를 설치합니다.

In [30]:
!pip install pytesseract

[다음 사이트][1]에서 OCR 프로그램을 다운로드 받습니다.

[1] : https://github.com/UB-Mannheim/tesseract/wiki